In [53]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from glob import glob
%matplotlib inline
import matplotlib.pyplot as plt
import pydicom

##Import any other stats/DL/ML packages you may need here. E.g. Keras, scikit-learn, etc.
from itertools import chain
from random import sample 

# Install a pip package in the current Jupyter kernel
import sys
# !{sys.executable} -m pip install opencv-python
# !{sys.executable} -m pip install --upgrade scikit-learn
from cv2 import cv2

import sklearn.preprocessing
import sklearn.model_selection

import skimage

import tensorflow as tf
import tensorflow.keras.layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

In [57]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filenames):
    dicom_data = []
    for img_name in filenames:
        print('Load file {} ...'.format(img_name))
        dcm = pydicom.dcmread(img_name)
        fields = [dcm.PatientID, int(dcm.PatientAge), dcm.PatientSex, dcm.Modality, dcm.StudyDescription, dcm.Rows, dcm.Columns, img_name, dcm.pixel_array]
        dicom_data.append(fields)
    dicom_df = pd.DataFrame(dicom_data, columns = ['PatientID','PatientAge','PatientSex','Modality','Finding Labels','Rows','Columns', 'Filename', 'Image'])
    print(dicom_df)
    return dicom_df
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input

def clahe_scaling(image):
        clahe = cv2.createCLAHE(clipLimit=3, tileGridSize=(16, 16))
        return clahe.apply(image)

def preprocess_image(src, IMG_SIZE=(224,224), INPUT_SHAPE=(1, 224,224,3)):
#     '''Takes an input image and returns a modified version of it'''
    dst = cv2.resize(src, IMG_SIZE) / 255
    dst = skimage.img_as_ubyte(dst)
    dst = clahe_scaling(dst)
    dst = skimage.img_as_float(dst)
    dst = cv2.cvtColor(dst.astype('float32'), cv2.COLOR_GRAY2RGB ) * 255
    return np.broadcast_to(dst, INPUT_SHAPE)

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # load model
    model = tensorflow.keras.models.load_model(model_path)
    model.load_weights(weight_path)
    # summarize model.
    print(model.summary())
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    prediction = my_model.predict(img, verbose = True)
    return prediction > thresh

In [58]:
my_model = load_model(model_path='my_model', weight_path='xray_class_my_model.best.hdf5')

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
preprocessing (Lambda)       (None, 224, 224, 3)       0         
_________________________________________________________________
model_1 (Functional)         (None, 2048)              23587712  
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dense_5 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_6 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 257       
Total params: 26,342,273
Trainable params: 2,754,561
Non-trainable params: 23,587,712
__________________________________

In [61]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']
dicom_df = check_dicom(test_dicoms)

model_path = 'my_model.json'
weight_path = 'xray_class_my_model.best.hdf5'

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
# img_mean = # loads the mean image value they used during training preprocessing
# img_std = # loads the std dev image value they used during training preprocessing

# my_model = load_model(model_path, weight_path)
my_model.load_weights(weight_path)
thresh = 0.6544955

# use the .dcm files to test your prediction
for index, img_data in dicom_df.iterrows():
    img = img_data['Image']
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img)
    pred = predict_image(my_model,img_proc,thresh)
    print(f'Predicted: {pred}, findings: {img_data["Finding Labels"]}')


Load file test1.dcm ...
Load file test2.dcm ...
Load file test3.dcm ...
Load file test4.dcm ...
Load file test5.dcm ...
Load file test6.dcm ...
  PatientID  PatientAge PatientSex Modality Finding Labels  Rows  Columns  \
0         2          81          M       DX     No Finding  1024     1024   
1         1          58          M       DX   Cardiomegaly  1024     1024   
2        61          77          M       DX       Effusion  1024     1024   
3         2          81          M       DX     No Finding  1024     1024   
4         2          81          M       CT     No Finding  1024     1024   
5         2          81          M       DX     No Finding  1024     1024   

    Filename                                              Image  
0  test1.dcm  [[199, 175, 152, 133, 124, 118, 113, 111, 110,...  
1  test2.dcm  [[202, 199, 195, 193, 195, 194, 193, 192, 184,...  
2  test3.dcm  [[142, 142, 143, 141, 143, 140, 140, 136, 137,...  
3  test4.dcm  [[199, 175, 152, 133, 124, 118, 113, 1